This bit is from last year's challenge, when we still had to do the retrieval on our own. It has lost some of its relevance since the organizers are doing the retrieval during submission. But still it saves valuable GPU time when evaluating MAP scores while training.

[FAISS](https://github.com/facebookresearch/faiss) from Facebook Research is a superfast package specialized for similarity search. The notebook will show how to do retrieval with faiss. It will also evaluate the difference in speeds between the scipy based search in the organizer's script and faiss based search and check how much time can you save in validation and training MAP score computations.

In [ ]:
import os
import cv2
import time
import numpy as np
from keras.models import Model
from keras.applications import VGG16,ResNet50
from scipy.spatial import distance

In [ ]:
# a dummy model, good old resnet50
model = ResNet50(weights='imagenet')

# removing the final classification layer 
model = Model(inputs=[model.input], outputs=[model.layers[-2].output])

Let's see how many images we have in our hand.

In [ ]:
index_files = []
for dirname, _, filenames in os.walk('../input/landmark-retrieval-2020/index/'):
    for filename in filenames:
        index_files.append(os.path.join(dirname, filename))

query_files = []
for dirname, _, filenames in os.walk('../input/landmark-retrieval-2020/test/'):
    for filename in filenames:
        query_files.append(os.path.join(dirname, filename))

print("We have a total of {} query images and a total of {} index images".format(len(query_files), len(index_files)))

Getting the embeddings from a subset of the query and index images

In [ ]:
index_embeddings = []
query_embeddings = []

# considering only the first 10,000 index images
for j,index_file in enumerate(index_files[:10000]):
    im = cv2.imread(index_file)
    im = cv2.resize(im, (224,224))
    index_embedding = model.predict(np.array([im]))[0]
    index_embeddings.append(index_embedding)

# considering only the first 500 query images
for i,query_file in enumerate(query_files[:500]):
    im = cv2.imread(query_file)
    im = cv2.resize(im, (224,224))
    query_embedding = model.predict(np.array([im]))[0]
    query_embeddings.append(query_embedding)
        

Timing the scipy method used in organizer's script. Retrieving top 100 samples from the index images for each query image.

In [ ]:
start = time.time()
distances = distance.cdist(np.array(query_embeddings), np.array(index_embeddings), 'euclidean')
predicted_positions = np.argpartition(distances, 100, axis=1)[:,:100]

print(predicted_positions.shape)
print("Time taken {} secs".format(time.time() - start))


Moving on to faiss!

In [ ]:
# install
!pip install faiss

import faiss                                 # make faiss available
faiss_index = faiss.IndexFlatL2(2048)        # build the index, need to input embedding size (last layer dimension of our model)
print(faiss_index.is_trained)

Now timing. Retrieving top 100 samples from the index images for each query image.

In [ ]:
# adding the index embeddings to faiss
faiss_index.add(np.array(index_embeddings))

# check how many are added
print("total embeddings added", faiss_index.ntotal) 

# now timing retrieval
start = time.time()
_, I = faiss_index.search(np.array(query_embeddings), 100)
    
print(I.shape)
print("Time taken {} secs".format(time.time() - start))
    

So a whopping ~30x improvement which will only get bigger with increasing number of index and query images.

This is taken from my last year's solutions which are available at https://github.com/mayukh18/Google-Landmark-Recognition-Retrieval-2019. Leave an upvote if you find these useful. :)